# Идея проекта, выборка и предобработка данных
<hr>

## Введение

```Текущая accuracy: 60-70%```

Проект представляет из себя анализатор дипфейков на основе анализа артефактов в различных зонах лица

В ходе предварительного анализа предметной области было выделено 3 наиболее перспективных для реализации метода анализа дипфейков:

- анализ движений глаз (саккад)
- анализ синзронизации речи и движений губ
- анализ артефактов

Так как анализ движений возможен только в динамике и довольно сложен в реализации, было решено реализовать метод анализа артефактов на статических изображениях (фото)

- В качестве "источника вдохновения" использовался [проект](https://github.com/rakshitsakhuja/Detecting-Deepfakes-with-OpenCV/blob/master/1.%20Processing%20Videos%20and%20Face%20Detection.ipynb), доступный по ссылке: 

https://github.com/rakshitsakhuja/Detecting-Deepfakes-with-OpenCV/blob/master/1.%20Processing%20Videos%20and%20Face%20Detection.ipynb

- В качестве датасета использовались фрагменты датасета [Deep Fake Detection Challenge DFDC](https://www.kaggle.com/competitions/deepfake-detection-challenge):

https://www.kaggle.com/competitions/deepfake-detection-challenge

- Про [распознанвание зон лица](https://pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/) можно прочесть по ссылке:

https://pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/

- Сжатый [датасет 300 Faces In-the-Wild Challenge (300-W)](https://ibug.doc.ic.ac.uk/resources/300-W/) для распознавания точек на лице и аннотации к нему:

https://ibug.doc.ic.ac.uk/resources/300-W/

## Пайплайн проекта

Проект работает в следующем порядке:

- datacollector.py, faceparts.py:
1. Считывание по 1 кадру из видео-датасета DFDC (5 x 10 GB, 5 x 1700 видео)
2. Распознавание 68 точек на лице с помощью OpenCV, DLib и датасета 300-W
3. Отсев фото, которые не удалось распознать (остается около 3500 фото), запись *размера* лица и бинарного значения яркости *пикселей* в 68 зонах вокруг ключевых точек в их черно-белой версии (размер зон - 8-17% от ширины лица, сейчас подбирается) в dataset (5 х 0.2-0.5 GB)
> псевдокод: ```image.to_greyscale().cut([zone_schape])```

- notebook.ipynb (до метки "Legacy"):
1. Добавление гистограмм распределения яркости к склейке датасетов
2. Добавление метрик std и noise для каждой зоны
3. Очистка от исходных массивов пикселей, запись на диск (30 MB)

- notebook2.ipynb:
1. Отсекание лишних фейков из исходного датасета (остается 1000=500*2 из 3500 фото) для избавления от перекоса (по крайней мере в процессе подбора методов, такой датасет удобнее, иначе недо/переобучение вощникает слишком часто)
2. Сбор std и avg значений для каждого из классов => определение точек, с наибольшим значением ```|avg_fake-avg_real| / (std_fake + std_real)```
> sklearn.feature_selection.SelectKBest работает плохо, т.к. не включает метрики для всего лица, которые нужны для нормализации значений по точкам

3. Выборка самых показательных features + features по всему лицу
4. Обучение моделей

> Так как 5 из 8 моделей дают сопоставимую точность, на данном этапе целесообразно работать надо подбором фичей и размерами сканируемых зон, мб фильтрацией плохих/маленьких фото, тюнинг моделей (тщательный) следует проводть позднее

## Потенциальные темы НИРов

> 1 тема != 1 НИР, темы могут быть объединены или разбиты

- процесс сборки данных: введение про проблему и методы анализа дипфейков, датасет DFDC, распознавание точек, OpenCV, DLib, датасет 300-W
- Предобработка данных: фильтрация, гистограммы и агрегационные метрики
- Подбор features, метрики метрик, метрики по всему лицу
- Выбор и тюнинг моделей, влияние размера и перекоса датасета и т.д.

На более поздних этапах (опционально):
- выстраивание вокруг модели эконсистемы: веб-приложение, комплект клиентов на React-native / чат-бот, Google OAuth, Metamask Auth
- Биллинг для приложений, ценообразование, токены (можно крипту из тестнета (Goerli, Ether testnet) ради забавы прикрутить), мониторинг (Grafana?), Docker-Kuber, балансировка и прочие умные штуки

# Обработка данных
<hr>

> 990 фото, баланс классов 1:1, размер зон - 8..17% от ширины лица (в процессе экспериментов), f1-метрика на разных моделях - 0.60..0.70

## Импорт dataset-а и агрегация данных для классов 'fake' и 'real'

In [1]:
import os
os.getcwd()

'C:\\Users\\sergey.astakhov\\Desktop\\BmstuDeepFake'

In [2]:
import pandas as pd
import numpy as np

In [32]:
df = pd.read_json("../deep_fake_src/dfdc_dataframes/df_total_0_4_compact_frame_10.json")
# df = pd.read_json("../deep_fake_src/dfdc_dataframes/dataframe_compact_total_27_frame_10_lim100_offset0.json")

In [33]:
# Балансировка классов 1:1
# Инверсные метрики для полиномов

# df = df.drop(columns=['index','face_size_px'])
df = pd.concat([
    df[df.fake==True].sample(
        int(df[df.fake==False].shape[0]*1)), 
    df[df.fake==False]
])

face = df.filter(like='face', axis=1).copy()

for col in face:
    name = str(col) + '_reversed'
    df[name] = face[str(col)].map(lambda x: 1.0 / max(1,x))

df = df.filter(regex='^(.(?!(var)))*$', axis=1).filter(regex='^(.(?!(noise_1)))*$', axis=1)

print(df.shape)
# df = df.reset_index()
df.head()

(990, 281)


,index,filename,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
1543,747,fnyrvnfegv.mp4,True,50176,18.980440,26.292957,17.768092,18.390150,15.467657,15.318861,...,30.000000,19.000000,34.000000,26.000000,0.456393,0.000020,0.017808,0.005533,1.0,1.0
1519,723,kthmgbrozc.mp4,True,131044,26.669729,13.890012,8.745546,15.384721,14.439757,17.578473,...,20.987654,18.518519,14.506173,9.876543,0.187723,0.000008,0.021741,0.001978,1.0,1.0
3339,721,qegmssagyr.mp4,True,28900,25.625229,39.587670,38.698148,52.319024,57.485458,55.969385,...,14.062500,7.812500,4.687500,9.375000,0.695502,0.000035,0.019415,0.007027,1.0,1.0
1005,209,gawakklefk.mp4,True,142884,16.945773,34.568434,21.086363,29.763092,16.663123,25.708218,...,9.259259,3.703704,5.864198,7.098765,0.168668,0.000007,0.020847,0.001952,1.0,1.0
214,224,frqhofmoob.mp4,True,70756,16.243801,24.723330,23.833235,29.718295,21.761410,35.450082,...,20.138889,8.333333,10.416667,36.111111,0.351914,0.000014,0.017746,0.003848,1.0,1.0


In [34]:
# агрегация для фейков

df_fakes_compact = df[df.fake==True].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_fakes_reduced = pd.DataFrame(df_fakes_compact.mean()).T
df_fakes_reduced['fake'] = True
df_fakes_reduced = df_fakes_reduced.set_index('fake')
df_fakes_reduced.filter(like='face', axis=1)

,face_size_px,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,,,
True,100067.418182,46.044214,525.427793,0.170235,0.27685,0.000015,0.024121,0.002982,0.999021,0.997693


In [35]:
# агрегация для реальных фото

df_real_compact = df[df.fake==False].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_real_reduced = pd.DataFrame(df_real_compact.mean()).T
df_real_reduced['fake'] = False
df_real_reduced = df_real_reduced.set_index('fake')
df_real_reduced.filter(like='face', axis=1)

,face_size_px,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,,,
False,92126.343434,49.182392,534.771606,0.200743,0.308146,0.000016,0.022526,0.003057,0.999456,0.998436


In [36]:
# самые ярко-различающиеся по классам нfeatures

df_compare = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_compare['diff_rel'] = abs(df_compare[False] - df_compare[True]) / (df_compare[False] + df_compare[True])
df_compare['diff'] = abs(df_compare[False] - df_compare[True]) 

df_compare = df_compare.sort_values(by=['diff_rel'], ascending=False)
df_compare.head(15)

fake,False,True,diff_rel,diff
index,541.650505,410.296970,0.137984,131.353535
pt_57_hist_noise_3,16.263845,12.338465,0.137240,3.925380
pt_11_hist_noise_0.5,1.231487,1.579773,0.123890,0.348287
pt_56_hist_noise_3,15.815474,12.350132,0.123035,3.465342
pt_24_hist_noise_0.5,1.649462,1.308693,0.115197,0.340769
pt_7_hist_noise_3,14.351787,11.422614,0.113647,2.929173
pt_58_hist_noise_3,17.602637,14.143828,0.108951,3.458809
pt_39_std,19.970445,16.108402,0.107045,3.862042
pt_57_std,13.351159,10.813399,0.105020,2.537760
pt_5_hist_noise_0.5,1.686957,2.071890,0.102407,0.384933


In [37]:
# самые стабильные features

df_fakes_std = pd.DataFrame(df_fakes_compact.std()).T
df_fakes_std['fake'] = True
df_fakes_std = df_fakes_std.set_index('fake')

df_real_std = pd.DataFrame(df_real_compact.std()).T
df_real_std['fake'] = False
df_real_std = df_real_std.set_index('fake')

df_std = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_std['total_std'] = (df_std[False] + df_std[True])

df_std = df_std.sort_values(by=['total_std'], ascending=True)
df_std.head(10)

fake,False,True,total_std
face_size_px_reversed,0.000016,0.000015,0.000030
overall_face_hist_std_reversed,0.003057,0.002982,0.006039
overall_face_std_reversed,0.022526,0.024121,0.046647
overall_face_hist_noise_0.5,0.200743,0.170235,0.370978
overall_face_hist_noise_3,0.308146,0.276850,0.584996
pt_8_hist_noise_0.5,0.507715,0.569635,1.077349
pt_7_hist_noise_0.5,0.540935,0.637549,1.178485
pt_9_hist_noise_0.5,0.687613,0.714655,1.402268
pt_6_hist_noise_0.5,0.688294,0.783802,1.472097
pt_57_hist_noise_0.5,0.871193,0.713770,1.584963


In [38]:
# самые надежные (разилчие_по_классам / разброс) features
# (var не кажется действительно надежной, скорее всего просто зависит от std квадратично или как-то еще)

df_smart = pd.concat([pd.DataFrame(df_compare['diff']),pd.DataFrame(df_std['total_std'])], axis=1)
df_smart['metric'] = df_smart['diff'] / df_smart['total_std']
df_smart = df_smart.sort_values(by=['metric'], ascending=False)
df_smart.head(30)

,diff,total_std,metric
index,131.353535,951.947475,0.137984
pt_57_hist_noise_3,3.925380,28.602310,0.137240
pt_11_hist_noise_0.5,0.348287,2.811260,0.123890
pt_56_hist_noise_3,3.465342,28.165606,0.123035
pt_24_hist_noise_0.5,0.340769,2.958155,0.115197
pt_7_hist_noise_3,2.929173,25.774401,0.113647
pt_58_hist_noise_3,3.458809,31.746464,0.108951
pt_39_std,3.862042,36.078847,0.107045
pt_57_std,2.537760,24.164559,0.105020
pt_5_hist_noise_0.5,0.384933,3.758846,0.102407


## Подбор features [to be modified]

> опирается на интуитивные предположения + метод определения фейков по шуму

In [39]:
features = list(set(
    list(df_smart.filter(like='face_hist', axis=0).index) +  # for relative metrics
    list(df_smart.filter(like='noise_0.5', axis=0).head(7).index) +
    list(df_smart.filter(like='noise_3', axis=0).head(7).index) +
    list(df_smart.filter(like='std', axis=0).head(5).index) +
    list(df_smart.filter(like='reversed', axis=0).index) 
    # list(df_smart.filter(like='var', axis=0).head(3).index) 
))

print(len(features))
features

27


['overall_face_hist_std_reversed',
 'pt_3_hist_noise_3',
 'pt_24_std',
 'pt_11_hist_noise_0.5',
 'overall_face_hist_noise_0.5',
 'pt_56_hist_noise_3',
 'pt_1_std',
 'face_size_px_reversed',
 'overall_face_hist_std',
 'pt_10_hist_noise_0.5',
 'pt_7_hist_noise_3',
 'overall_face_hist_noise_0.5_reversed',
 'pt_30_hist_noise_0.5',
 'pt_57_hist_noise_0.5',
 'pt_24_hist_noise_0.5',
 'pt_58_hist_std',
 'pt_58_hist_noise_3',
 'overall_face_std_reversed',
 'pt_5_hist_noise_0.5',
 'pt_39_std',
 'pt_57_std',
 'pt_57_hist_noise_3',
 'pt_21_hist_noise_0.5',
 'overall_face_hist_noise_3',
 'pt_45_hist_noise_3',
 'overall_face_hist_noise_3_reversed',
 'pt_39_hist_noise_3']

## Разбивка данных на train и test сплиты

In [40]:
# очистка dataset-а от лишних колонок перед тренировкой модели (необязательно?)

df_compact = df.filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
df_compact.head(5)

,index,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
1543,747,True,50176,18.980440,26.292957,17.768092,18.390150,15.467657,15.318861,14.509566,...,30.000000,19.000000,34.000000,26.000000,0.456393,0.000020,0.017808,0.005533,1.0,1.0
1519,723,True,131044,26.669729,13.890012,8.745546,15.384721,14.439757,17.578473,22.042034,...,20.987654,18.518519,14.506173,9.876543,0.187723,0.000008,0.021741,0.001978,1.0,1.0
3339,721,True,28900,25.625229,39.587670,38.698148,52.319024,57.485458,55.969385,44.148011,...,14.062500,7.812500,4.687500,9.375000,0.695502,0.000035,0.019415,0.007027,1.0,1.0
1005,209,True,142884,16.945773,34.568434,21.086363,29.763092,16.663123,25.708218,15.809500,...,9.259259,3.703704,5.864198,7.098765,0.168668,0.000007,0.020847,0.001952,1.0,1.0
214,224,True,70756,16.243801,24.723330,23.833235,29.718295,21.761410,35.450082,27.587431,...,20.138889,8.333333,10.416667,36.111111,0.351914,0.000014,0.017746,0.003848,1.0,1.0


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [42]:
y = df_compact['fake']
X = pd.DataFrame(df_compact.drop(columns=['fake']))
# selector = SelectKBest(f_classif, k=7)
# X = selector.fit_transform(X, y)
X =  df_compact[features]

# mask = selector.get_support()
# X = X[X.columns[mask]]

# df_compact.filter(like='size', axis=0).head()

# print(df_compact[X.columns[mask]].shape)
# df_compact[X.columns[mask]].head()

pd.DataFrame(X).head()

,overall_face_hist_std_reversed,pt_3_hist_noise_3,pt_24_std,pt_11_hist_noise_0.5,overall_face_hist_noise_0.5,pt_56_hist_noise_3,pt_1_std,face_size_px_reversed,overall_face_hist_std,pt_10_hist_noise_0.5,...,overall_face_std_reversed,pt_5_hist_noise_0.5,pt_39_std,pt_57_std,pt_57_hist_noise_3,pt_21_hist_noise_0.5,overall_face_hist_noise_3,pt_45_hist_noise_3,overall_face_hist_noise_3_reversed,pt_39_hist_noise_3
1543,0.005533,48.000000,24.642151,0.000000,0.281011,24.000000,9.050144,0.000020,180.736103,0.000000,...,0.017808,0.000000,10.331505,10.492073,18.000000,0.000000,0.456393,23.000000,1.0,20.000000
1519,0.001978,10.802469,14.586359,3.395062,0.119044,5.864198,4.155099,0.000008,505.479641,3.703704,...,0.021741,0.617284,17.702121,4.158222,4.938272,1.234568,0.187723,10.185185,1.0,17.901235
3339,0.007027,3.125000,18.798687,0.000000,0.449827,26.562500,4.498236,0.000035,142.312554,0.000000,...,0.019415,0.000000,51.695161,8.881160,17.187500,0.000000,0.695502,65.625000,1.0,68.750000
1005,0.001952,25.308642,12.172597,1.234568,0.097282,5.246914,37.324169,0.000007,512.342563,0.000000,...,0.020847,4.938272,15.666636,6.992255,2.160494,3.086420,0.168668,21.604938,1.0,18.209877
214,0.003848,34.722222,12.625877,0.000000,0.224716,18.055556,33.542319,0.000014,259.879156,0.000000,...,0.017746,0.000000,25.351350,7.384524,10.416667,0.000000,0.351914,23.611111,1.0,35.416667


In [43]:
# print(df_compact.filter(like='size', axis=1).shape)
# df_compact.filter(like='size', axis=1).head()

In [44]:
# sizes_col = X.filter(like='size', axis=1).copy()
# X = pd.concat([
#     df_compact[X.columns[mask]],
#     sizes_col
# ], axis=0)
# # X.join(df_compact.filter(like='size', axis=1), lsuffix='_caller', rsuffix='_other')
# X.head()

In [45]:
X_train, X_test, y_train, y_test = train_test_split( 
    X, 
    y,
    test_size=0.20, 
    random_state=420)

In [46]:
pd.DataFrame(X_train).head()

,overall_face_hist_std_reversed,pt_3_hist_noise_3,pt_24_std,pt_11_hist_noise_0.5,overall_face_hist_noise_0.5,pt_56_hist_noise_3,pt_1_std,face_size_px_reversed,overall_face_hist_std,pt_10_hist_noise_0.5,...,overall_face_std_reversed,pt_5_hist_noise_0.5,pt_39_std,pt_57_std,pt_57_hist_noise_3,pt_21_hist_noise_0.5,overall_face_hist_noise_3,pt_45_hist_noise_3,overall_face_hist_noise_3_reversed,pt_39_hist_noise_3
1874,0.000908,17.750000,27.099132,4.750000,0.059993,3.000000,16.133186,0.000005,1101.052972,1.250000,...,0.017269,2.250000,16.051814,6.773839,3.000000,1.750000,0.100873,16.500000,1.0,13.000000
497,0.000976,2.160494,11.011883,0.308642,0.051790,2.777778,4.314655,0.000007,1024.197531,0.000000,...,0.040677,0.308642,4.400477,3.458602,2.777778,0.617284,0.096582,3.086420,1.0,3.395062
2185,0.003152,2.000000,11.944271,0.000000,0.072874,5.000000,7.914613,0.000021,317.272840,0.000000,...,0.041762,0.000000,12.195815,4.669679,2.000000,0.000000,0.220765,19.000000,1.0,19.000000
1051,0.002161,2.083333,14.987532,0.000000,0.086641,3.472222,3.354073,0.000013,462.788968,0.000000,...,0.027427,0.000000,9.070303,6.343681,8.333333,0.000000,0.196912,19.444444,1.0,11.111111
2329,0.002955,29.296875,10.069751,6.250000,0.152439,12.890625,28.787787,0.000009,338.385216,0.390625,...,0.014984,6.250000,24.731900,12.582340,13.281250,1.171875,0.223081,12.500000,1.0,26.562500


In [47]:
y_train.head()

1874    False
497      True
2185     True
1051     True
2329    False
Name: fake, dtype: bool

## Обучение моделей

### Служебные функции

In [48]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier


In [49]:
import joblib

In [50]:
def save_model(model, X_train, y_train, X_test, y_test, p=2, name='my_model.pkl'):
    clf = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=p), 
        model
    )
    clf.fit(X_train, y_train)
    joblib.dump(clf, name)

In [51]:
def train_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    clf = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=p), 
        model
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return (
        confusion_matrix(y_test, y_pred), 
        classification_report(
            y_test, 
            y_pred, 
            target_names=['class "real"', 'class "fakes"'], 
            zero_division=np.nan)
    )

In [52]:
def print_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    confusion_matrix, classification_report = train_polynomial_pipeline(
        model, 
        X_train, 
        y_train, 
        X_test, 
        y_test,
        p
    )
    print(confusion_matrix)
    print(classification_report)

### SGDClassifier (f1=0.60) [to be modified]

In [53]:
print_polynomial_pipeline(
    SGDClassifier(),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[59 42]
 [48 49]]
               precision    recall  f1-score   support

 class "real"       0.55      0.58      0.57       101
class "fakes"       0.54      0.51      0.52        97

     accuracy                           0.55       198
    macro avg       0.54      0.54      0.54       198
 weighted avg       0.55      0.55      0.54       198



### SVC (f1=0.64) [to be modified]

In [54]:
print_polynomial_pipeline(
    SVC(gamma='auto'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[57 44]
 [31 66]]
               precision    recall  f1-score   support

 class "real"       0.65      0.56      0.60       101
class "fakes"       0.60      0.68      0.64        97

     accuracy                           0.62       198
    macro avg       0.62      0.62      0.62       198
 weighted avg       0.62      0.62      0.62       198



### LogisticRegression (f1=0.61) [to be modified]

In [55]:
print_polynomial_pipeline(
    # LogisticRegression(max_iter=15000)
    LogisticRegression(
        max_iter=15000, 
        # penalty=None,
        class_weight='balanced',
        solver='liblinear',
        # tol=1e-6
    ),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[63 38]
 [43 54]]
               precision    recall  f1-score   support

 class "real"       0.59      0.62      0.61       101
class "fakes"       0.59      0.56      0.57        97

     accuracy                           0.59       198
    macro avg       0.59      0.59      0.59       198
 weighted avg       0.59      0.59      0.59       198



### NN - MLPClassifier (f1=0.66) [to be modified]

In [56]:
print_polynomial_pipeline(
    # MLPClassifier(max_iter=5000), # 0.57
    MLPClassifier(
        solver='lbfgs', 
        hidden_layer_sizes=(34,), # 5, 7, 7, 3 - 0.60
        random_state=1, 
        alpha=0.001, 
        # activation='relu',
        tol=1e-6,
        max_fun=15000,
        max_iter=15000),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
) 

[[59 42]
 [40 57]]
               precision    recall  f1-score   support

 class "real"       0.60      0.58      0.59       101
class "fakes"       0.58      0.59      0.58        97

     accuracy                           0.59       198
    macro avg       0.59      0.59      0.59       198
 weighted avg       0.59      0.59      0.59       198



### RandomForestClassifier (good, f1=0.64) [to be modified] 

In [57]:
print_polynomial_pipeline(
    RandomForestClassifier(max_depth=7, random_state=42),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 
# save_model(
#     RandomForestClassifier(max_depth=7, random_state=42),
#     X_train, 
#     y_train, 
#     X_test, 
#     y_test,
#     p=2,
#     name="random_forest_10.pkl"
# ) 

[[61 40]
 [31 66]]
               precision    recall  f1-score   support

 class "real"       0.66      0.60      0.63       101
class "fakes"       0.62      0.68      0.65        97

     accuracy                           0.64       198
    macro avg       0.64      0.64      0.64       198
 weighted avg       0.64      0.64      0.64       198



### DecisionTreeClassifier (f1=0.59) [to be modified]

In [58]:
print_polynomial_pipeline(
    DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
)

[[42 59]
 [23 74]]
               precision    recall  f1-score   support

 class "real"       0.65      0.42      0.51       101
class "fakes"       0.56      0.76      0.64        97

     accuracy                           0.59       198
    macro avg       0.60      0.59      0.57       198
 weighted avg       0.60      0.59      0.57       198



### RadiusNeighborsClassifier (f1=0.46) [уязвим к выбросам]

In [59]:
print_polynomial_pipeline(
    RadiusNeighborsClassifier(
        radius=100, 
        weights='distance', 
        p=1, 
        outlier_label='most_frequent'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
)

[[32 69]
 [17 80]]
               precision    recall  f1-score   support

 class "real"       0.65      0.32      0.43       101
class "fakes"       0.54      0.82      0.65        97

     accuracy                           0.57       198
    macro avg       0.59      0.57      0.54       198
 weighted avg       0.60      0.57      0.54       198



### KNeighborsClassifier (f1=0.69) [to be modified]

In [60]:
print_polynomial_pipeline(
    KNeighborsClassifier(n_neighbors=6, weights='distance'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
)

[[62 39]
 [32 65]]
               precision    recall  f1-score   support

 class "real"       0.66      0.61      0.64       101
class "fakes"       0.62      0.67      0.65        97

     accuracy                           0.64       198
    macro avg       0.64      0.64      0.64       198
 weighted avg       0.64      0.64      0.64       198

